In [2]:
import json
from lib2to3.pgen2 import driver
from macpath import join
from operator import contains
from turtle import title
from webbrowser import get
from selenium import webdriver
from selenium.webdriver import ChromeOptions
import time
import re
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import uuid
import os
import requests
import pandas as pd
import demjson3
import jsonpath
import urllib.request 
import urllib.parse

c:\Users\huancwang2\.conda\envs\regular\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: the macpath module is deprecated in 3.7 and will be removed in 3.8
  This is separate from the ipykernel package so we can avoid doing imports until


# 可以用 .replace(r'[^\u4e00-\u9fa5]','') 来处理Unicode编码问题

In [62]:
Art_dict = {2:'传统音乐', 3:'传统舞蹈', 4:'传统戏剧', 5:'曲艺', 7:'传统美术'}
num_total = {2:431, 3:356, 4:473, 5:213, 7:417}
list_art_ich = []

In [63]:

for ich_type in Art_dict.keys():
    url = 'https://www.ihchina.cn/Article/Index/getProject.html?'
    total = num_total[ich_type]
    data = {
        'province':'',
        'rx_time':'',
        'type':ich_type,
        'cate':'',
        'keywords':'',
        'category_id':16,
        'limit':total,
        'p':1
    }

    data = urllib.parse.urlencode(data)
    url = url + data

    header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'}

    request = urllib.request.Request(url=url, headers=header)
    print(url)
    response = urllib.request.urlopen(request)
    json_each_type = demjson3.decode(response.read().decode('utf-8'))
    info = jsonpath.jsonpath(json_each_type, '$.list[*]')
    list_art_ich.append(info)

https://www.ihchina.cn/Article/Index/getProject.html?province=&rx_time=&type=2&cate=&keywords=&category_id=16&limit=431&p=1
response
json
https://www.ihchina.cn/Article/Index/getProject.html?province=&rx_time=&type=3&cate=&keywords=&category_id=16&limit=356&p=1
response
json
https://www.ihchina.cn/Article/Index/getProject.html?province=&rx_time=&type=4&cate=&keywords=&category_id=16&limit=473&p=1
response
json
https://www.ihchina.cn/Article/Index/getProject.html?province=&rx_time=&type=5&cate=&keywords=&category_id=16&limit=213&p=1
response
json
https://www.ihchina.cn/Article/Index/getProject.html?province=&rx_time=&type=7&cate=&keywords=&category_id=16&limit=417&p=1
response
json


In [72]:
flat_dict = list_art_ich[0] + list_art_ich[1] + list_art_ich[2] + list_art_ich[3] + list_art_ich[4]

In [251]:
df_total_ich_art = pd.DataFrame(flat_dict)

In [275]:
df_total_ich_art = pd.read_csv('./json_Art_list/df_total_ich_art.csv')

In [195]:
df_total_ich_art.columns

Index(['status_id', 'status', 'is_top', 'top_start', 'top_end', 'sortrank',
       'modified', 'id', 'num', 'auto_id', 'order', 'title', 'title2',
       'source', 'type', 'reg_type', 'num_first', 'num_sec', 'cate',
       'project_num', 'rx_time', 'address', 'province', 'city', 'area',
       'project_level', 'content', 'playtype', 'upvideo', 'img_group', 'video',
       'unit', 'protect_unit', 'img_video'],
      dtype='object')

In [276]:
drop_list = []
df_total_ich_art = df_total_ich_art.drop(columns=['status_id', 'status', 'is_top', 'top_start', 'top_end', 'sortrank', 'modified', 'title2', 'source', 'reg_type', 'project_level', 'upvideo', 'img_group', 'video', 'unit', 'img_video'])

KeyError: "['status_id' 'status' 'is_top' 'top_start' 'top_end' 'sortrank' 'modified'\n 'title2' 'source' 'reg_type' 'project_level' 'upvideo' 'img_group'\n 'video' 'unit' 'img_video'] not found in axis"

id:唯一索引\
num:编号 = (num_first + num_sec)\
auto_id:项目序号\
project_num的结构：num(编号)+index(同名项目中的顺序下标)+area(细化到县) 
rx_time:加入时间（批数）
province:省市县
地区编号结构：address:地区号（细化到省）
                city: 细化到城市
                    area：细化到县
unit == province

In [277]:
df_total_ich_art['province'] = df_total_ich_art['province'].apply(lambda x: x.strip())
df_total_ich_art.loc[df_total_ich_art['content']=='', 'content'] = 'N'+ df_total_ich_art.loc[df_total_ich_art['content']=='', 'province']+'None'
df_total_ich_art['content'] = df_total_ich_art['content'].apply(lambda x: x.strip())

In [278]:
df_total_ich_art

,id,num,auto_id,order,title,type,num_first,num_sec,cate,project_num,rx_time,address,province,city,area,content,playtype,protect_unit
0,12408,Ⅱ-1,32,NaN,左权开花调,传统音乐,2,1,新增项目,Ⅱ-001_01_140722,"2006,(第一批)",140000.0,山西省左权县,140700.0,140722.0,申报地区或单位：山西省左权县 左权县位于太行山主脉西侧、山西省东部边缘，古称辽州、辽县，后...,local,左权县民歌研究中心
1,12409,Ⅱ-2,33,NaN,河曲民歌,传统音乐,2,2,新增项目,Ⅱ-002_01_140930,"2006,(第一批)",140000.0,山西省河曲县,140900.0,140930.0,申报地区或单位：山西省河曲县 河曲位于山西省西北部，东界偏关、五寨，南界岢岚、保德，西、北...,local,河曲县二人台艺术研究中心
2,12410,Ⅱ-3,34,NaN,蒙古族长调民歌,传统音乐,2,3,新增项目,Ⅱ-003_01_150000,"2006,(第一批)",150000.0,内蒙古自治区,NaN,NaN,申报地区或单位：内蒙古自治区 长调是蒙古语“乌日汀哆”的意译。“乌日汀”为“长久”、“永恒...,local,内蒙古自治区艺术研究所（内蒙古自治区艺术档案馆）
3,12411,Ⅱ-3,34,NaN,蒙古族长调民歌,传统音乐,2,3,扩展项目,Ⅱ-003_02_652800,"2008,(第二批)",650000.0,新疆维吾尔自治区巴音郭楞蒙古自治州,652800.0,NaN,申报地区或单位：新疆维吾尔自治区巴音郭楞蒙古自治州 长调系蒙古语“乌日汀哆”的意译，它是北...,local,新疆巴音郭楞蒙古自治州文化馆
4,12412,Ⅱ-3,34,NaN,蒙古族长调民歌,传统音乐,2,3,扩展项目,Ⅱ-003_02_654226,"2008,(第二批)",650000.0,新疆维吾尔自治区和布克赛尔蒙古自治县,654200.0,654226.0,申报地区或单位：新疆维吾尔自治区和布克赛尔蒙古自治县 长调系蒙古语“乌日汀哆”的意译，它是...,local,和布克赛尔蒙古自治县非物质文化遗产保护中心
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,23550,Ⅶ-135,1483,NaN,瓷刻（大丰瓷刻）,传统美术,7,135,新增项目,NaN,"2021,(第五批)",320000.0,江苏省盐城市大丰区,320900.0,320982.0,N江苏省盐城市大丰区None,local,盐城市大丰区瓷刻世家工艺厂
1886,23551,Ⅶ-136,1484,NaN,竹根雕（象山竹根雕）,传统美术,7,136,新增项目,NaN,"2021,(第五批)",330000.0,浙江省宁波市象山县,330200.0,330225.0,N浙江省宁波市象山县None,local,象山德和根艺美术馆
1887,23552,Ⅶ-137,1485,NaN,贝雕（北海贝雕）,传统美术,7,137,新增项目,NaN,"2021,(第五批)",450000.0,广西壮族自治区北海市,450500.0,NaN,N广西壮族自治区北海市None,local,北海市恒兴珠宝有限责任公司
1888,23553,Ⅶ-138,1486,NaN,骨角雕（合浦角雕）,传统美术,7,138,新增项目,NaN,"2021,(第五批)",450000.0,广西壮族自治区北海市合浦县,450500.0,450521.0,N广西壮族自治区北海市合浦县None,local,合浦金蝠角雕厂


In [282]:
df_total_ich_art['rx_time'] = df_total_ich_art['rx_time'].apply(lambda x: x.replace('</br>', ','))
df_total_ich_art['content'] = df_total_ich_art['content'].apply(lambda x: x.replace('&lt;br /&gt;\r\n\u3000\u3000', ''))
df_total_ich_art['content'] = df_total_ich_art['content'].apply(lambda x: x.replace('&lt;br /&gt;', ''))
df_total_ich_art['content'] = df_total_ich_art['content'].apply(lambda x: x.replace('&lt;span style=&quot;white-space:normal;&quot;&gt;', ''))
df_total_ich_art['true_content'] = df_total_ich_art.apply(lambda x: x['content'].split(x['province'][-3:],1)[1], axis=1)
df_total_ich_art

,id,num,auto_id,order,title,type,num_first,num_sec,cate,project_num,rx_time,address,province,city,area,content,playtype,protect_unit,true_content
0,12408,Ⅱ-1,32,NaN,左权开花调,传统音乐,2,1,新增项目,Ⅱ-001_01_140722,"2006,(第一批)",140000.0,山西省左权县,140700.0,140722.0,申报地区或单位：山西省左权县 左权县位于太行山主脉西侧、山西省东部边缘，古称辽州、辽县，后...,local,左权县民歌研究中心,左权县位于太行山主脉西侧、山西省东部边缘，古称辽州、辽县，后因纪念牺牲在此的抗日名将而改...
1,12409,Ⅱ-2,33,NaN,河曲民歌,传统音乐,2,2,新增项目,Ⅱ-002_01_140930,"2006,(第一批)",140000.0,山西省河曲县,140900.0,140930.0,申报地区或单位：山西省河曲县 河曲位于山西省西北部，东界偏关、五寨，南界岢岚、保德，西、北...,local,河曲县二人台艺术研究中心,河曲位于山西省西北部，东界偏关、五寨，南界岢岚、保德，西、北隔黄河分别与陕西、内蒙相邻，...
2,12410,Ⅱ-3,34,NaN,蒙古族长调民歌,传统音乐,2,3,新增项目,Ⅱ-003_01_150000,"2006,(第一批)",150000.0,内蒙古自治区,NaN,NaN,申报地区或单位：内蒙古自治区 长调是蒙古语“乌日汀哆”的意译。“乌日汀”为“长久”、“永恒...,local,内蒙古自治区艺术研究所（内蒙古自治区艺术档案馆）,长调是蒙古语“乌日汀哆”的意译。“乌日汀”为“长久”、“永恒”之意，“哆”为“歌”之意。...
3,12411,Ⅱ-3,34,NaN,蒙古族长调民歌,传统音乐,2,3,扩展项目,Ⅱ-003_02_652800,"2008,(第二批)",650000.0,新疆维吾尔自治区巴音郭楞蒙古自治州,652800.0,NaN,申报地区或单位：新疆维吾尔自治区巴音郭楞蒙古自治州 长调系蒙古语“乌日汀哆”的意译，它是北...,local,新疆巴音郭楞蒙古自治州文化馆,长调系蒙古语“乌日汀哆”的意译，它是北方草原游牧民族在畜牧业生产劳动中创造的一种民歌，在...
4,12412,Ⅱ-3,34,NaN,蒙古族长调民歌,传统音乐,2,3,扩展项目,Ⅱ-003_02_654226,"2008,(第二批)",650000.0,新疆维吾尔自治区和布克赛尔蒙古自治县,654200.0,654226.0,申报地区或单位：新疆维吾尔自治区和布克赛尔蒙古自治县 长调系蒙古语“乌日汀哆”的意译，它是...,local,和布克赛尔蒙古自治县非物质文化遗产保护中心,长调系蒙古语“乌日汀哆”的意译，它是北方草原游牧民族在畜牧业生产劳动中创造的一种民歌，在...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,23550,Ⅶ-135,1483,NaN,瓷刻（大丰瓷刻）,传统美术,7,135,新增项目,NaN,"2021,(第五批)",320000.0,江苏省盐城市大丰区,320900.0,320982.0,N江苏省盐城市大丰区None,local,盐城市大丰区瓷刻世家工艺厂,None
1886,23551,Ⅶ-136,1484,NaN,竹根雕（象山竹根雕）,传统美术,7,136,新增项目,NaN,"2021,(第五批)",330000.0,浙江省宁波市象山县,330200.0,330225.0,N浙江省宁波市象山县None,local,象山德和根艺美术馆,None
1887,23552,Ⅶ-137,1485,NaN,贝雕（北海贝雕）,传统美术,7,137,新增项目,NaN,"2021,(第五批)",450000.0,广西壮族自治区北海市,450500.0,NaN,N广西壮族自治区北海市None,local,北海市恒兴珠宝有限责任公司,None
1888,23553,Ⅶ-138,1486,NaN,骨角雕（合浦角雕）,传统美术,7,138,新增项目,NaN,"2021,(第五批)",450000.0,广西壮族自治区北海市合浦县,450500.0,450521.0,N广西壮族自治区北海市合浦县None,local,合浦金蝠角雕厂,None


In [287]:
df_total_ich_art['content'] = df_total_ich_art['true_content']
df_total_ich_art = df_total_ich_art.drop(columns=['true_content'])
df_total_ich_art['num'] = df_total_ich_art['num'].str.strip()
df_total_ich_art.to_csv('./json_Art_list/df_total_ich_art_Final.csv', index=False)

In [263]:
df_total_ich_art.to_csv('./json_Art_list/df_total_ich_art.csv', index=False, encoding='utf-8')

In [3]:
df_total_ich_art = pd.read_csv('./json_Art_list/df_total_ich_art_Final.csv')

In [6]:
df_total_ich_art

,id,num,auto_id,order,title,type,num_first,num_sec,cate,project_num,rx_time,address,province,city,area,content,playtype,protect_unit
0,12408,Ⅱ-1,32,NaN,左权开花调,传统音乐,2,1,新增项目,Ⅱ-001_01_140722,"2006,(第一批)",140000.0,山西省左权县,140700.0,140722.0,左权县位于太行山主脉西侧、山西省东部边缘，古称辽州、辽县，后因纪念牺牲在此的抗日名将而改...,local,左权县民歌研究中心
1,12409,Ⅱ-2,33,NaN,河曲民歌,传统音乐,2,2,新增项目,Ⅱ-002_01_140930,"2006,(第一批)",140000.0,山西省河曲县,140900.0,140930.0,河曲位于山西省西北部，东界偏关、五寨，南界岢岚、保德，西、北隔黄河分别与陕西、内蒙相邻，...,local,河曲县二人台艺术研究中心
2,12410,Ⅱ-3,34,NaN,蒙古族长调民歌,传统音乐,2,3,新增项目,Ⅱ-003_01_150000,"2006,(第一批)",150000.0,内蒙古自治区,NaN,NaN,长调是蒙古语“乌日汀哆”的意译。“乌日汀”为“长久”、“永恒”之意，“哆”为“歌”之意。...,local,内蒙古自治区艺术研究所（内蒙古自治区艺术档案馆）
3,12411,Ⅱ-3,34,NaN,蒙古族长调民歌,传统音乐,2,3,扩展项目,Ⅱ-003_02_652800,"2008,(第二批)",650000.0,新疆维吾尔自治区巴音郭楞蒙古自治州,652800.0,NaN,长调系蒙古语“乌日汀哆”的意译，它是北方草原游牧民族在畜牧业生产劳动中创造的一种民歌，在...,local,新疆巴音郭楞蒙古自治州文化馆
4,12412,Ⅱ-3,34,NaN,蒙古族长调民歌,传统音乐,2,3,扩展项目,Ⅱ-003_02_654226,"2008,(第二批)",650000.0,新疆维吾尔自治区和布克赛尔蒙古自治县,654200.0,654226.0,长调系蒙古语“乌日汀哆”的意译，它是北方草原游牧民族在畜牧业生产劳动中创造的一种民歌，在...,local,和布克赛尔蒙古自治县非物质文化遗产保护中心
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,23550,Ⅶ-135,1483,NaN,瓷刻（大丰瓷刻）,传统美术,7,135,新增项目,NaN,"2021,(第五批)",320000.0,江苏省盐城市大丰区,320900.0,320982.0,None,local,盐城市大丰区瓷刻世家工艺厂
1886,23551,Ⅶ-136,1484,NaN,竹根雕（象山竹根雕）,传统美术,7,136,新增项目,NaN,"2021,(第五批)",330000.0,浙江省宁波市象山县,330200.0,330225.0,None,local,象山德和根艺美术馆
1887,23552,Ⅶ-137,1485,NaN,贝雕（北海贝雕）,传统美术,7,137,新增项目,NaN,"2021,(第五批)",450000.0,广西壮族自治区北海市,450500.0,NaN,None,local,北海市恒兴珠宝有限责任公司
1888,23553,Ⅶ-138,1486,NaN,骨角雕（合浦角雕）,传统美术,7,138,新增项目,NaN,"2021,(第五批)",450000.0,广西壮族自治区北海市合浦县,450500.0,450521.0,None,local,合浦金蝠角雕厂
